# Scripts para crear los archivos txt con los inserts

Importanción de librerías:

In [165]:
import pandas as pd
import numpy as np
from itertools import chain
from database import Database

db = Database('../db/fifa25.db')

Ruta global y lectura de los datasets

In [166]:
BASE_PATH = './../docs/'
df_females = pd.read_csv(f'{BASE_PATH}female_players.csv', sep=',')
df_males = pd.read_csv(f'{BASE_PATH}male_players.csv', sep=',')
df_all = pd.read_csv(f'{BASE_PATH}all_players.csv', sep=',')
df_all.columns

/tmp/ipykernel_33613/3131349356.py:4: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv(f'{BASE_PATH}all_players.csv', sep=',')


Index(['id', 'Rank', 'Name', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY',
       'Acceleration', 'Sprint Speed', 'Positioning', 'Finishing',
       'Shot Power', 'Long Shots', 'Volleys', 'Penalties', 'Vision',
       'Crossing', 'Free Kick Accuracy', 'Short Passing', 'Long Passing',
       'Curve', 'Dribbling', 'Agility', 'Balance', 'Reactions', 'Ball Control',
       'Composure', 'Interceptions', 'Heading Accuracy', 'Def Awareness',
       'Standing Tackle', 'Sliding Tackle', 'Jumping', 'Stamina', 'Strength',
       'Aggression', 'Position', 'Weak foot', 'Skill moves', 'Preferred foot',
       'Height', 'Weight', 'Alternative positions', 'Age', 'Nation', 'League',
       'Team', 'play style', 'url', 'GK Diving', 'GK Handling', 'GK Kicking',
       'GK Positioning', 'GK Reflexes', 'Unnamed: 57'],
      dtype='object')

#### Llenar play_styles

Vamos a leer los <b>play style</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE play_styles (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  name	VARCHAR(30)
);
```

In [167]:
rows = df_all['play style'].str.split(',')
play_styles = []
for array_str in rows:
    if isinstance(array_str, list):
        for style in array_str:
            # remove '* '
            if style[0] == ' ':
                style = style[1:]
            # remove ' *'
            if style[len(style) - 1] == ' ':
                style = style[:-1]
            if style not in play_styles:
                play_styles.append(style)
text = '-- migrate:up \n\n'
i = 1
for style in play_styles:
    text = text + f"INSERT INTO play_styles (id, name) VALUES ({i}, '{style}');\n"
    i = i + 1
text = text + '\n-- migrate:down \n\nDELETE FROM play_styles;'
with open(f'{BASE_PATH}play_styles_inserts.sql', 'w') as archivo:
    archivo.write(text)

#### Llenar nations

Vamos a leer los <b>natios</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE nations (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  name	VARCHAR(40)
);
```

In [168]:
nations = sorted(list(df_all['Nation'].unique()))
text = '-- migrate:up \n\n'
i = 1
for name in nations:
    text += "INSERT INTO nations (id, name) VALUES (" + str(i) + ", '" + name + "');\n"
    i += 1
text += '\n-- migrate:down \n\nDELETE FROM nations;'

with open(f'{BASE_PATH}nations_inserts.sql', 'w') as archivo:
    archivo.write(text)

#### Llenar positions

Vamos a leer los <b>positions</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE positions (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  name	VARCHAR(40)
);
```

In [169]:
# 'Position'  'Alternative positions'

positions = sorted(list(df_all['Position'].unique()))
positions
alternative_positions = df_all['Alternative positions']

rows = df_all['Alternative positions'].str.split(',')
for array_str in rows:
    if isinstance(array_str, list):
        for position in array_str:
            # remove '* '
            if position[0] == ' ':
                position = position[1:]
            # remove ' *'
            if position[len(position) - 1] == ' ':
                position = position[:-1]
            if position not in positions:
                positions.append(position)

text = '-- migrate:up \n\n'
i = 1
for name in positions:
    text += "INSERT INTO positions (id, name) VALUES (" + str(i) + ", '" + name.replace("'", "''") + "');\n"
    i = i + 1
text = text + '\n-- migrate:down \n\nDELETE FROM positions;'
with open(f'{BASE_PATH}positions_inserts.sql', 'w') as archivo:
    archivo.write(text)
    

#### Llenar leagues

Vamos a leer los <b>leagues</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE leagues (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  name	VARCHAR(30),
  nation_id	INTEGER,
  FOREIGN KEY (nation_id) REFERENCES nations (id)
);
```

In [170]:
df_leagues_nations = pd.read_csv(f'{BASE_PATH}league_nation.csv', sep=',')
leagues = sorted(list(df_all['League'].unique()))
text = '-- migrate:up \n\n'
i = 1
for index, row in df_leagues_nations.iterrows():
    rs = db.fetchone(f"SELECT id FROM nations WHERE name = '{row['Nation']}'")
    if rs is not None:
        nation_id = rs['id']
        name = row['League']
        text += "INSERT INTO leagues (id, name, nation_id) VALUES (" + str(i) + ", '" + name.replace("'", "''") + "', " + str(nation_id) + ");\n"
        i = i + 1
        #print(f"Index: {index}, League: {row['League']}, Nation: {row['Nation']}, NationID: {nation_id}")
    else:
        print(f"Nation '{row['Nation']}' not found in the database.")
text = text + '\n-- migrate:down \n\nDELETE FROM leagues;'
with open(f'{BASE_PATH}leagues_inserts.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text)

Conexión establecida.


#### Llenar teams

Vamos a leer los <b>teams</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE teams (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  name	VARCHAR(40),
  league_id	INTEGER NOT NULL,
  FOREIGN KEY (league_id) REFERENCES leagues (id)
);
```

In [171]:

teams_leagues = df_all[['id','Team', 'League']].to_numpy()
teams_list = []
for tl in teams_leagues:
  team = tl[1]
  league = tl[2].replace("'", "''")
  query = f"SELECT id FROM leagues WHERE name = '{league}'"
  rs = db.fetchone(query)
  i = 1
  if rs is not None:
    league_id = rs['id']
    if len(str(league_id)) == 1:
      league_id = '0' + str(league_id)
    else:
      league_id = str(league_id)
    tmp = league_id + '::' + team + '::' + league
    if tmp not in teams_list:
      teams_list.append(tmp)
  else:
      print(f"Leage '{league}' not found in the database.")
teams_list.sort()
# inserts
i = 1
text = '-- migrate:up \n\n'
for t in teams_list:
  data_list = t.split('::')
  team = data_list[1]
  league = data_list[2]
  league_id = int(data_list[0])
  text += "INSERT INTO teams (id, name, league_id) VALUES (" + str(i) + ", '" + team.replace("'", "''") + "', " + str(league_id) + ");\n"
  i = i + 1
text = text + '\n-- migrate:down \n\nDELETE FROM teams;'
with open(f'{BASE_PATH}teams_inserts.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text)


### Llenar Jugadores

Vamos a leer los <b>teams</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE players (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  name	VARCHAR(60),
  rank INTEGER,
  weak_foot INTEGER,
  skill_moves INTEGER,
  heigth INTEGER,
  weight INTEGER,
  url VARCHAR(120),
  foot_id INTEGER NOT NULL,
  sex_id INTEGER NOT NULL,
  position_id INTEGER NOT NULL,
  nation_id	INTEGER NOT NULL,
  team_id	INTEGER NOT NULL,
  FOREIGN KEY (foot_id) REFERENCES foots (id),
  FOREIGN KEY (sex_id) REFERENCES sexs (id),
  FOREIGN KEY (position_id) REFERENCES positions (id),
  FOREIGN KEY (nation_id) REFERENCES nations (id)
  FOREIGN KEY (team_id) REFERENCES teams (id)
);
```

In [172]:
male_playes_data = df_males[['Name', 'Rank', 'Weak foot', 'Skill moves', 'Height', 'Weight', 'url', 'Preferred foot', 'Position', 'Nation', 'Team', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Age', 'url2']].to_numpy()
female_playes_data = df_females[['Name', 'Rank', 'Weak foot', 'Skill moves', 'Height', 'Weight', 'url', 'Preferred foot', 'Position', 'Nation', 'Team', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Age', 'url2']].to_numpy()

text_players = '-- migrate:up \n\n'
player_id = 1
# fill males
for player in male_playes_data:
  # player data
  name = player[0].replace("'", "''")
  rank = player[1]
  weak_foot = player[2]
  skill_moves = player[3]
  height = player[4].split(' / ')[0][:-2]
  weight = player[5].split(' / ')[0][:-2]
  url = player[6] + player[24]
  foot_id = 1 if player[7] == 'Right' else 2
  sex_id = 1
  position = player[8]
  position_id = 0
  nation = player[9].replace("'", "''")
  nation_id = 0
  team = player[10].replace("'", "''")
  team_id = 0
  age = player[23]
  # common data
  overall = player[11]
  velocity = player[12] # PAC = velocidad
  shooting = player[13]
  passing = player[14]
  dribbling = player[15]
  defending = player[16]
  physicality = player[17]
  # gk data
  diving = player[18]
  handling = player[19]
  kicking = player[20]
  positioning = player[21]
  reflexes = player[22]
  # position_id
  query = f"SELECT id FROM positions WHERE name = '{position}'"
  rs = db.fetchone(query)
  id = 1
  if rs is not None:
    position_id = rs['id']
  else:
    print(f"PoCôte d''Ivoiresition '{position}' not found in the database.")
  # nation_id
  query = f"SELECT id FROM nations WHERE name = '{nation}'"
  rs = db.fetchone(query)
  if rs is not None:
    nation_id = rs['id']
  else:
    if nation == "Côte d''Ivoire":
      nation_id = 39
    elif nation == "Curaçao":
      nation_id = 36
    else:
      print(f"Nation '{nation}' not found in the database.")
  # team_id
  query = f"SELECT id FROM teams WHERE name = '{team}'"
  rs = db.fetchone(query)
  if rs is not None:
    team_id = rs['id']
  else:
    print(f"Team '{team}' not found in the database.")
  # insert player
  text_players += "INSERT INTO players (id, name, rank, weak_foot, skill_moves, heigth, weight, url, foot_id, sex_id, position_id, nation_id, team_id, age) VALUES (" + str(player_id) + ", '" + name.replace("'", "''") + "', " + str(rank) + ", " + str(weak_foot) + ', ' + str(skill_moves) + ', ' + str(height) + ', ' + str(weight) + ", '" + url + "', " + str(foot_id) + ', ' + str(sex_id) + ', ' + str(position_id) + ', ' + str(nation_id) + ', ' + str(team_id) + ', ' + str(age) + ");\n"

  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
  player_id = player_id + 1
# fill females
for player in female_playes_data:
  # player data
  name = player[0].replace("'", "''")
  rank = player[1]
  weak_foot = player[2]
  skill_moves = player[3]
  height = player[4].split(' / ')[0][:-2]
  weight = player[5].split(' / ')[0][:-2]
  url = player[6] + player[24]
  foot_id = 1 if player[7] == 'Right' else 2
  sex_id = 2
  position = player[8]
  position_id = 0
  nation = player[9].replace("'", "''")
  nation_id = 0
  team = player[10].replace("'", "''")
  team_id = 0
  age = player[23]
  # common data
  overall = player[11]
  velocity = player[12] # PAC = velocidad
  shooting = player[13]
  passing = player[14]
  dribbling = player[15]
  defending = player[16]
  physicality = player[17]
  # gk data
  diving = player[18]
  handling = player[19]
  kicking = player[20]
  positioning = player[21]
  reflexes = player[22]
  # position_id
  query = f"SELECT id FROM positions WHERE name = '{position}'"
  rs = db.fetchone(query)
  id = 1
  if rs is not None:
    position_id = rs['id']
  else:
    print(f"PoCôte d''Ivoiresition '{position}' not found in the database.")
  # nation_id
  query = f"SELECT id FROM nations WHERE name = '{nation}'"
  rs = db.fetchone(query)
  if rs is not None:
    nation_id = rs['id']
  else:
    if nation == "Côte d''Ivoire":
      nation_id = 39
    elif nation == "Curaçao":
      nation_id = 36
    else:
      print(f"Nation '{nation}' not found in the database.")
  # team_id
  query = f"SELECT id FROM teams WHERE name = '{team}'"
  rs = db.fetchone(query)
  if rs is not None:
    team_id = rs['id']
  else:
    print(f"Team '{team}' not found in the database.")
  # insert player
  text_players += "INSERT INTO players (id, name, rank, weak_foot, skill_moves, heigth, weight, url, foot_id, sex_id, position_id, nation_id, team_id, age) VALUES (" + str(player_id) + ", '" + name.replace("'", "''") + "', " + str(rank) + ", " + str(weak_foot) + ', ' + str(skill_moves) + ', ' + str(height) + ', ' + str(weight) + ", '" + url + "', " + str(foot_id) + ', ' + str(sex_id) + ', ' + str(position_id) + ', ' + str(nation_id) + ', ' + str(team_id) + ', ' + str(age) + ");\n"

  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
  player_id = player_id + 1
# write male an female
text_players = text_players + '\n-- migrate:down \n\nDELETE FROM players;'
with open(f'{BASE_PATH}players_inserts.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text_players)

### Llenar Common Details

Vamos a leer los <b>common details</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE common_details (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  overall INTEGER,
  velocity INTEGER,
  shooting INTEGER,
  passing INTEGER,
  dribbling INTEGER,
  defending INTEGER,
  physicality INTEGER,
  player_id	INTEGER NOT NULL,
  FOREIGN KEY (player_id) REFERENCES players (id)
);
```

In [174]:
male_playes_data = df_males[['Name', 'Rank', 'Weak foot', 'Skill moves', 'Height', 'Weight', 'url', 'Preferred foot', 'Position', 'Nation', 'Team', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Age', 'url2']].to_numpy()
female_playes_data = df_females[['Name', 'Rank', 'Weak foot', 'Skill moves', 'Height', 'Weight', 'url', 'Preferred foot', 'Position', 'Nation', 'Team', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Age', 'url2']].to_numpy()

text_players = '-- migrate:up \n\n'
player_id = 1
id = 1
# fill males
for player in male_playes_data:
  # player data
  position = player[8]
  # common data
  overall = player[11]
  velocity = player[12] # PAC = velocidad
  shooting = player[13]
  passing = player[14]
  dribbling = player[15]
  defending = player[16]
  physicality = player[17]
  # position_id
  # insert player common
  text_players += "INSERT INTO common_details (id, overall, velocity, shooting, passing, dribbling, defending, physicality, player_id) VALUES (" + str(id) + ', ' + str(overall) + ', ' + str(velocity) + ', ' + str(shooting) + ', ' + str(passing) + ', ' + str(dribbling) + ', ' + str(defending) + ', ' + str(physicality) + ', ' + str(player_id) + ");\n"
  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
  player_id = player_id + 1
  id = id + 1
# fill females
for player in female_playes_data:
  position = player[8]
  # common data
  overall = player[11]
  velocity = player[12] # PAC = velocidad
  shooting = player[13]
  passing = player[14]
  dribbling = player[15]
  defending = player[16]
  physicality = player[17]
  # position_id
  # insert player common
  text_players += "INSERT INTO common_details (id, overall, velocity, shooting, passing, dribbling, defending, physicality, player_id) VALUES (" + str(id) + ', ' + str(overall) + ', ' + str(velocity) + ', ' + str(shooting) + ', ' + str(passing) + ', ' + str(dribbling) + ', ' + str(defending) + ', ' + str(physicality) + ', ' + str(player_id) + ");\n"
  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
  player_id = player_id + 1
  id = id + 1

  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
  player_id = player_id + 1
# write male an female
text_players = text_players + '\n-- migrate:down \n\nDELETE FROM common_details;'
with open(f'{BASE_PATH}common_details_inserts.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text_players)

### Llenar Goalkeepers Details

Vamos a leer los <b>goalkeepers details</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE goalkeeper_details (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  diving INTEGER,
  handling INTEGER,
  kicking INTEGER,
  positioning INTEGER,
  reflexes INTEGER,
  player_id	INTEGER NOT NULL,
  FOREIGN KEY (player_id) REFERENCES players (id)
);
```

In [176]:
male_playes_data = df_males[['Name', 'Rank', 'Weak foot', 'Skill moves', 'Height', 'Weight', 'url', 'Preferred foot', 'Position', 'Nation', 'Team', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Age', 'url2']].to_numpy()
female_playes_data = df_females[['Name', 'Rank', 'Weak foot', 'Skill moves', 'Height', 'Weight', 'url', 'Preferred foot', 'Position', 'Nation', 'Team', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Age', 'url2']].to_numpy()

text_players = '-- migrate:up \n\n'
player_id = 1
id = 1
# fill males
for player in male_playes_data:
  # player data
  position = player[8]
  # common data
  # gk data
  diving = player[18]
  handling = player[19]
  kicking = player[20]
  positioning = player[21]
  reflexes = player[22]
  # position_id
  # insert player common
  if position == 'GK':
    text_players += "INSERT INTO goalkeeper_details (id, diving, handling, kicking, positioning, reflexes, player_id) VALUES (" + str(id) + ', ' + str(diving) + ', ' + str(handling) + ', ' + str(kicking) + ', ' + str(positioning) + ', ' + str(reflexes) + ', ' + str(player_id) + ");\n"
  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
  player_id = player_id + 1
  id = id + 1
# fill females
for player in female_playes_data:
  position = player[8]
  # common data
  # gk data
  diving = player[18]
  handling = player[19]
  kicking = player[20]
  positioning = player[21]
  reflexes = player[22]
  # position_id
  # insert player common
  if position == 'GK':
    text_players += "INSERT INTO goalkeeper_details (id, diving, handling, kicking, positioning, reflexes, player_id) VALUES (" + str(id) + ', ' + str(diving) + ', ' + str(handling) + ', ' + str(kicking) + ', ' + str(positioning) + ', ' + str(reflexes) + ', ' + str(player_id) + ");\n"
  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
  player_id = player_id + 1
  id = id + 1

  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
  player_id = player_id + 1
# write male an female
text_players = text_players + '\n-- migrate:down \n\nDELETE FROM goalkeeper_details;'
with open(f'{BASE_PATH}goalkeeper_details_inserts.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text_players)

### Llenar players_play_styles

Vamos a leer los <b>players_play_styles</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE players_play_styles (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  play_style_id	INTEGER NOT NULL,
  player_id	INTEGER NOT NULL,
  FOREIGN KEY (play_style_id) REFERENCES play_styles (id),
  FOREIGN KEY (player_id) REFERENCES players (id)
);
```

In [185]:
male_playes_data = df_males[['play style']].to_numpy()
female_playes_data = df_females[['play style']].to_numpy()

text_players = '-- migrate:up \n\n'
player_id = 1
id = 1
# fill males
for player in male_playes_data:
  if not pd.isna(player[0]):
    styles = player[0].split(', ')
    for s in styles:
      # play_style_id
      query = f"SELECT id FROM play_styles WHERE name = '{s}'"
      rs = db.fetchone(query)
      if rs is not None:
        play_style_id = rs['id']
        text_players += "INSERT INTO players_play_styles (id, play_style_id, player_id) VALUES (" + str(id) + ', ' + str(play_style_id) + ', ' + str(player_id) + ");\n"
        id = id + 1
      else:
        print(f"PlayStylePlayer '{s}' not found in the database.")
  player_id = player_id + 1
# fill females
for player in female_playes_data:
  if not pd.isna(player[0]):
    styles = player[0].split(', ')
    for s in styles:
      # play_style_id
      query = f"SELECT id FROM play_styles WHERE name = '{s}'"
      rs = db.fetchone(query)
      if rs is not None:
        play_style_id = rs['id']
        text_players += "INSERT INTO players_play_styles (id, play_style_id, player_id) VALUES (" + str(id) + ', ' + str(play_style_id) + ', ' + str(player_id) + ");\n"
        id = id + 1
      else:
        print(f"PlayStylePlayer '{s}' not found in the database.")
  player_id = player_id + 1
# write male an female
text_players = text_players + '\n-- migrate:down \n\nDELETE FROM players_play_styles;'
with open(f'{BASE_PATH}players_play_styles_inserts.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text_players)

### Llenar players_positions

Vamos a leer los <b>players_positions</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE players_positions (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  position_id	INTEGER NOT NULL,
  player_id	INTEGER NOT NULL,
  FOREIGN KEY (position_id) REFERENCES positions (id),
  FOREIGN KEY (player_id) REFERENCES players (id)
);
```

In [186]:
male_playes_data = df_males[['Alternative positions']].to_numpy()
female_playes_data = df_females[['Alternative positions']].to_numpy()

text_players = '-- migrate:up \n\n'
player_id = 1
id = 1
# fill males
for player in male_playes_data:
  if not pd.isna(player[0]):
    styles = player[0].split(', ')
    for s in styles:
      # play_style_id
      query = f"SELECT id FROM positions WHERE name = '{s}'"
      rs = db.fetchone(query)
      if rs is not None:
        position_id = rs['id']
        text_players += "INSERT INTO players_positions (id, position_id, player_id) VALUES (" + str(id) + ', ' + str(position_id) + ', ' + str(player_id) + ");\n"
        id = id + 1
      else:
        print(f"Position '{s}' not found in the database.")
  player_id = player_id + 1
# fill females
for player in female_playes_data:
  if not pd.isna(player[0]):
    styles = player[0].split(', ')
    for s in styles:
      # position_id
      query = f"SELECT id FROM positions WHERE name = '{s}'"
      rs = db.fetchone(query)
      if rs is not None:
        position_id = rs['id']
        text_players += "INSERT INTO players_positions (id, position_id, player_id) VALUES (" + str(id) + ', ' + str(position_id) + ', ' + str(player_id) + ");\n"
        id = id + 1
      else:
        print(f"Position '{s}' not found in the database.")
  player_id = player_id + 1
# write male an female
text_players = text_players + '\n-- migrate:down \n\nDELETE FROM players_positions;'
with open(f'{BASE_PATH}players_positions_inserts.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text_players)